In [1]:
import json 
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, MaxPooling2D , LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

In [2]:
with open('intents.json') as file:
    data = json.load(file)

In [3]:
data

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi', 'Hey', 'Is anyone there?', 'Hello', 'Hay'],
   'responses': ['Hello', 'Hi', 'Hi there']},
  {'tag': 'goodbye',
   'patterns': ['Bye', 'See you later', 'Goodbye'],
   'responses': ['See you later', 'Have a nice day', 'Bye! Come back again']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Thanks for the help'],
   'responses': ['Happy to help!',
    'Any time!',
    'My pleasure',
    "You're most welcome!"]},
  {'tag': 'about',
   'patterns': ['Who are you?', 'What are you?', 'Who you are?'],
   'responses': ['I.m Joana, your bot assistant',
    "I'm Joana, an Artificial Intelligent bot"]},
  {'tag': 'name',
   'patterns': ['what is your name',
    'what should I call you',
    'whats your name?'],
   'responses': ['You can call me Joana.',
    "I'm Joana!",
    'Just call me as Joana']},
  {'tag': 'help',
   'patterns': ['Could you help me?',
    'give me a hand please',
    'Can you help

In [4]:
training_sentences = []
training_labels = []
labels = []
responses = []

In [5]:
(training_labels)

[]

In [6]:
for intent in data['intents']:
    for pattern in intent['patterns']:
        training_sentences.append(pattern)
        training_labels.append(intent['tag'])
    responses.append(intent['responses'])
    
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
num_classes = len(labels)

In [7]:
lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [8]:
training_labels

array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 0, 0, 6, 6, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1], dtype=int64)

In [9]:
from keras.preprocessing.text import one_hot


onehot_repr=[one_hot(words,1000)for words in training_sentences] 

In [10]:
from keras.preprocessing.sequence import pad_sequences
embedded_doc = pad_sequences(onehot_repr , padding = 'pre' , maxlen = 20)

In [11]:
np.shape(training_labels)

(33,)

In [12]:
from keras.utils import to_categorical
training_labels = to_categorical(training_labels)

In [13]:
model = Sequential()
model.add(Embedding(1000, 10, input_length=20))
model.add(LSTM(32))
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='sigmoid'))

model.compile(loss='categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 10)            10000     
_________________________________________________________________
lstm (LSTM)                  (None, 32)                5504      
_________________________________________________________________
dense (Dense)                (None, 16)                528       
_________________________________________________________________
dense_1 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 136       
Total params: 16,440
Trainable params: 16,440
Non-trainable params: 0
_________________________________________________________________


In [14]:
y = np.array(training_labels)
X = np.array(embedded_doc)

In [15]:
model.fit(X, y , epochs = 300)

Epoch 1/300
2/2 [==============================] - 4s 16ms/step - loss: 2.0805 - accuracy: 0.0919
Epoch 2/300
2/2 [==============================] - 0s 18ms/step - loss: 2.0784 - accuracy: 0.2449
Epoch 3/300
2/2 [==============================] - 0s 13ms/step - loss: 2.0776 - accuracy: 0.3674
Epoch 4/300
2/2 [==============================] - 0s 12ms/step - loss: 2.0772 - accuracy: 0.3368
Epoch 5/300
2/2 [==============================] - 0s 12ms/step - loss: 2.0769 - accuracy: 0.3062
Epoch 6/300
2/2 [==============================] - 0s 12ms/step - loss: 2.0763 - accuracy: 0.2143
Epoch 7/300
2/2 [==============================] - 0s 14ms/step - loss: 2.0760 - accuracy: 0.1225
Epoch 8/300
2/2 [==============================] - 0s 24ms/step - loss: 2.0759 - accuracy: 0.2345
Epoch 9/300
2/2 [==============================] - 0s 25ms/step - loss: 2.0757 - accuracy: 0.1531
Epoch 10/300
2/2 [==============================] - 0s 15ms/step - loss: 2.0751 - accuracy: 0.1531
Epoch 11/300
2/2 [=

In [16]:
ans = model.predict_classes(X)
ans

C:\Users\User-1\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([4, 4, 4, 4, 4, 3, 3, 3, 7, 7, 7, 7, 0, 4, 0, 6, 4, 6, 5, 5, 5, 5,
       5, 5, 5, 2, 2, 2, 2, 2, 1, 1, 1], dtype=int64)

In [17]:
li = []
def chat():
    while True:
            inp = [input()]
            if inp[0] == 'quit':
                break
            onehot_repr=[one_hot(words,1000)for words in inp]
            embedded_doc = pad_sequences(onehot_repr , padding = 'pre' , maxlen = 20)
            Xt = np.array(embedded_doc)
            res = model.predict(Xt)
            tag = lbl_encoder.inverse_transform([np.argmax(res)])
            for i in data['intents']:
                if i['tag'] == tag:
                    print(np.random.choice(i['responses']))

In [18]:
chat()

hi
Hello
what is your name
You can call me Joana.
i weant to have sex
Please provide us your complaint in order to assist you
someone is kidnapping me
Bye! Come back again
kk
Hi there
kk
Hi there
quit
